# Выбираем группы и разделяем на A/Б

Выберите две из следующих особенностей в данных и придумайте для них А/Б тест.

Surge => более низкая price-to-order конверсия  
Android => более высокие юзерские отмены  
Заказ из центра => более низкие юзерские отмены  
Comfort & Business классы => более низкие юзерские отмены, более длинные поездк  
Сформулируйте гипотезы по всем правилам  
Для описанных А/Б тестов разбейте юзеров на группы с помощью рандома  

Простой трек:  
1 Проверьте однородность с помощью t-теста, предполагая, что ЦПТ выполняется  

Средний трек:  
2 Проверьте однородность с помощью рангового критерия  

Сложный трек:  
3 Проверьте однородность с помощью бакетинга/бутстрапа  

Date - датасессии  
●User_id - id пользователя  
●Hour - час стартасессии  
●App_opened - приложение открыто  
●Price_seen - пользователь ввел данные маршрута и показана цена  
●Order_made - пользователь кликнул по кнопке заказа  
●Surge - в этот период был включен surge (надбавка к стоимости поездки в период повышенного спроса)  
●Ride_completed - поездка успешно завершена  
●User_cancelled - пользователь отменил поездку  
●Age - возраст пользователя  
●Os - операционная система на телефоне  
●City_center_order - заказ был сделан из центра города  
●Order_class - класс поездки  
●Distance - дистанция в км  
●Rfm - rfm-сегмент пользователя

In [5]:
import pandas as pd
import numpy as np
from scipy import stats as st
from scipy.stats import mannwhitneyu
from scipy.stats import norm
import random

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
df = pd.read_csv('Netology_A_B-testing_HW_1.csv')
df.head()

,Unnamed: 0,date,user_id,hour,app_opened,price_seen,order_made,surge,ride_completed,user_cancelled,age,os,city_center_order,order_class,distance,rfm
0,0,2020-05-21,867689,12,1,1,1,no surge,1,0,20,iOS,0,business,7.982135,low
1,1,2020-05-23,752172,5,1,1,1,no surge,1,0,37,Android,1,economy,2.908468,high
2,2,2020-05-20,486559,15,1,1,1,no surge,1,0,47,Android,0,comfort,7.224614,low
3,3,2020-05-19,304024,0,1,1,1,no surge,1,0,59,Android,1,economy,1.874349,low
4,4,2020-05-23,139420,0,1,1,1,no surge,1,0,19,Android,0,business,10.704778,medium


In [4]:
df.describe()

,Unnamed: 0,user_id,hour,app_opened,price_seen,order_made,ride_completed,user_cancelled,age,city_center_order,distance
count,101500.000000,101500.000000,101500.000000,101500.0,101500.000000,101500.000000,101500.000000,101500.000000,101500.000000,101500.000000,91431.000000
mean,50749.500000,549874.802207,11.481429,1.0,0.900798,0.731389,0.620365,0.111025,40.471764,0.571675,5.371152
std,29300.670499,258600.104176,6.917522,0.0,0.298934,0.443239,0.485299,0.314164,13.536322,0.494838,4.118531
min,0.000000,100093.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.010946
25%,25374.750000,328223.000000,5.000000,1.0,1.000000,0.000000,0.000000,0.000000,29.000000,0.000000,2.438335
50%,50749.500000,548827.000000,11.000000,1.0,1.000000,1.000000,1.000000,0.000000,40.000000,1.000000,4.279565
75%,76124.250000,773051.000000,17.000000,1.0,1.000000,1.000000,1.000000,0.000000,52.000000,1.000000,7.129814
max,101499.000000,999978.000000,23.000000,1.0,1.000000,1.000000,1.000000,1.000000,69.000000,1.000000,40.268966


**Surge => более низкая price-to-order конверсия**

H0: Конверсия заказов не отличается в моменты повышенного спроса и в обычное время

In [11]:
# разделим датафрейм по целевой метрике
df_surge = df[ (df['price_seen'] == 1) & (df['surge'] == 'surge') ]
df_nosurge = df[ (df['price_seen'] == 1) & (df['surge'] == 'no surge')]
print(len(df_surge))
print(len(df_nosurge))

# оставим только уникальных пользователей и усредним показатели
df_surge_uniq = df_surge.groupby(['user_id']).mean().reset_index()
df_nosurge_uniq = df_nosurge.groupby(['user_id']).mean().reset_index()
print(len(df_surge_uniq))
print(len(df_surge_uniq))

31643
59788
17780
17780


Для описанных А/Б тестов разбейте юзеров на группы с помощью рандома

In [20]:
# Для описанных А/Б тестов разбейте юзеров на группы с помощью рандома
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html

# в подвыборке оставим 3000 пользователей, т.к. для стат.тестов минимальный порог 2500

df_surge_uniq_sample = df_surge_uniq.sample(n=3000, random_state=1)
df_nosurge_uniq_sample = df_nosurge_uniq.sample(n=3000, random_state=1)

In [26]:
# проверим конверсию для выборки и подвыборки
print('конерсия полной выборки уникальных пользователей в часы пик', df_surge_uniq['order_made'].mean())
print('конерсия подвыборки 3000 уникальных пользователей в часы пик', df_surge_uniq_sample['order_made'].mean())

print('конерсия полной выборки уникальных пользователей в обычное время', df_nosurge_uniq['order_made'].mean())
print('конерсия подвыборки 3000 уникальных пользователей в обычное время', df_nosurge_uniq_sample['order_made'].mean())

конерсия полной выборки уникальных пользователей в часы пик 0.7235210107665058
конерсия подвыборки 3000 уникальных пользователей в часы пик 0.7357865079365088
конерсия полной выборки уникальных пользователей в обычное время 0.8598660942214601
конерсия подвыборки 3000 уникальных пользователей в обычное время 0.8628289682539685


Проверьте однородность с помощью t-теста, предполагая, что ЦПТ выполняется

In [21]:
result_tt = st.ttest_ind(df_surge_uniq_sample['order_made'], df_nosurge_uniq_sample['order_made'], equal_var=False)
alpha = 0.05
if (result_tt.pvalue < alpha):
    print(f'Отвергаем нулевую гипотезу, конверсия в часы пик отличается, statistic - {result_tt.statistic}  pvalue - {round(result_tt.pvalue, 5)}')
else:
    print(f'Не отвергаем нулевую гипотезу конверсия одинаковая, statistic - {result_tt.statistic}  pvalue - {round(result_tt.pvalue, 5)}')

Отвергаем нулевую гипотезу, конверсия в часы пик отличается, statistic - -15.81287648523105  pvalue - 0.0


statistic отрицательный, следовательно, конверсия в часы пик ниже

Проверьте однородность с помощью рангового критерия

In [22]:
result_mw = mannwhitneyu(df_surge_uniq_sample['order_made'], df_nosurge_uniq_sample['order_made'])
if result_mw.pvalue < alpha and result_mw.statistic > 0:
    print(f'конверсия различается. statistic - {result_mw.statistic} pvalue - {round(result_mw.pvalue, 5)}')
else:
    print(f'конверсия не различается. statistic - {result_mw.statistic} pvalue - {round(result_mw.pvalue, 5)}')

конверсия различается. statistic - 3880550.5 pvalue - 0.0


тест Манн-Уитни тоже показывает, что statistic отрицательный - конверсия в часы пик ниже

Проверьте однородность с помощью бакетинга/бутстрапа

бутстрап

In [24]:
means = []
for i in range(10000):
    surge = df_surge_uniq_sample.sample(frac=1, replace=True).order_made.mean()
    nosurge = df_nosurge_uniq_sample.sample(frac=1, replace=True).order_made.mean()
    means.append(nosurge - surge)
#расчет доверительного интервала
pd_means = pd.DataFrame(means)
confidence_interval = pd_means.quantile([0.025, 0.975])
#расчет p_value
p_1 = norm.cdf(x = 0, loc = np.mean(means), scale = np.std(means))
p_2 = norm.cdf(x = 0, loc = -np.mean(means), scale = np.std(means))
p_value = min(p_1, p_2) * 2
if p_value < alpha and list(confidence_interval[0])[0] > 0:
    print(f'конверсия в обычные часы выше, чем в часы пик. pvalue - {round(p_value, 5)}, доверительный интервал {[round(v,2) for v in confidence_interval[0].tolist()]}')
elif p_value < alpha and list(confidence_interval[0])[0] < 0:      
    print(f'конверсия в обычные часы ниже, чем в часы пик. pvalue - {round(p_value, 5)}, доверительный интервал {[round(v,2) for v in confidence_interval[0].tolist()]}')
else:
    print(f'конверсия не различается. pvalue - {round(p_value, 5)}, доверительный интервал {[round(v,2) for v in confidence_interval[0].tolist()]}')

конверсия в обычные часы выше, чем в часы пик. pvalue - 0.0, доверительный интервал [0.11, 0.14]


**Android => более высокие юзерские отмены**

H0: количество отказов не зависит от используемой операционной системы

In [32]:
# разделим датафрейм по целевой метрике
df_ios = df[ (df['os'] == 'iOS') & (df['price_seen'] == 1) ]
df_android = df[ (df['os'] == 'Android') & (df['price_seen'] == 1)]
print(len(df_ios))
print(len(df_android))

# оставим только уникальных пользователей и усредним показатели
df_ios_uniq = df_ios.groupby(['user_id']).mean().reset_index()
df_android_uniq = df_android.groupby(['user_id']).mean().reset_index()
print(len(df_ios_uniq))
print(len(df_android_uniq))

41382
50049
19988
21397


Для описанных А/Б тестов разбейте юзеров на группы с помощью рандома

In [33]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html

# в подвыборке оставим 3000 пользователей, т.к. для стат.тестов минимальный порог 2500

df_ios_uniq_sample = df_ios_uniq.sample(n=3000, random_state=1)
df_android_uniq_sample = df_android_uniq.sample(n=3000, random_state=1)

In [35]:
# проверим конверсию для выборки и подвыборки
print('конерсия полной выборки уникальных пользователей ios', df_ios_uniq['order_made'].mean())
print('конерсия подвыборки 3000 уникальных пользователей android', df_ios_uniq_sample['order_made'].mean())

print('конерсия полной выборки уникальных пользователей ios', df_android_uniq['order_made'].mean())
print('конерсия подвыборки 3000 уникальных пользователей android', df_android_uniq_sample['order_made'].mean())

конерсия полной выборки уникальных пользователей ios 0.8570517111854304
конерсия подвыборки 3000 уникальных пользователей android 0.8631492063492064
конерсия полной выборки уникальных пользователей ios 0.7759445397700613
конерсия подвыборки 3000 уникальных пользователей android 0.7817376984126978


Проверьте однородность с помощью t-теста, предполагая, что ЦПТ выполняется

In [36]:
result_tt = st.ttest_ind(df_ios_uniq_sample['user_cancelled'], df_android_uniq_sample['user_cancelled'], equal_var=False)
alpha = 0.05
if (result_tt.pvalue < alpha):
    print(f'Отвергаем нулевую гипотезу, количество отказов различается, statistic - {result_tt.statistic}  pvalue - {round(result_tt.pvalue, 5)}')
else:
    print(f'Не отвергаем нулевую гипотезу, отказы в обеих группах одинаковые, statistic - {result_tt.statistic}  pvalue - {round(result_tt.pvalue, 5)}')

Отвергаем нулевую гипотезу, количество отказов различается, statistic - -7.601923031380705  pvalue - 0.0


statistic отрицательный, следовательно, пользователи iOS совершают меньше отказов

Проверьте однородность с помощью рангового критерия

In [37]:
result_mw = mannwhitneyu(df_ios_uniq_sample['user_cancelled'], df_android_uniq_sample['user_cancelled'])
if result_mw.pvalue < alpha and result_mw.statistic > 0:
    print(f'конверсия различается. statistic - {result_mw.statistic} pvalue - {round(result_mw.pvalue, 5)}')
else:
    print(f'конверсия не различается. statistic - {result_mw.statistic} pvalue - {round(result_mw.pvalue, 5)}')

конверсия различается. statistic - 4001932.5 pvalue - 0.0


Проверьте однородность с помощью бакетинга/бутстрапа  
  
бутстрап

In [39]:
means = []
for i in range(10000):
    ios = df_ios_uniq_sample.sample(frac=1, replace=True).ride_completed.mean()
    android = df_android_uniq_sample.sample(frac=1, replace=True).ride_completed.mean()
    means.append(ios - android)
#расчет доверительного интервала
pd_means = pd.DataFrame(means)
confidence_interval = pd_means.quantile([0.025, 0.975])
#расчет p_value
p_1 = norm.cdf(x = 0, loc = np.mean(means), scale = np.std(means))
p_2 = norm.cdf(x = 0, loc = -np.mean(means), scale = np.std(means))
p_value = min(p_1, p_2) * 2
if p_value < alpha and list(confidence_interval[0])[0] > 0:
    print(f'пользователи iOS совершают меньше отказов. pvalue - {round(p_value, 5)}, доверительный интервал {[round(v,2) for v in confidence_interval[0].tolist()]}')
elif p_value < alpha and list(confidence_interval[0])[0] < 0:      
    print(f'пользователи Android совершают меньше отказов. pvalue - {round(p_value, 5)}, доверительный интервал {[round(v,2) for v in confidence_interval[0].tolist()]}')
else:
    print(f'количество отказов в обеих группах не зависит от ОС. pvalue - {round(p_value, 5)}, доверительный интервал {[round(v,2) for v in confidence_interval[0].tolist()]}')

пользователи iOS совершают меньше отказов. pvalue - 0.0, доверительный интервал [0.11, 0.15]
